In [1]:
%pip install eep153_tools
%pip install python_gnupg
%pip install -U gspread_pandas

  Using cached eep153_tools-0.12.4-py2.py3-none-any.whl.metadata (363 bytes)
Using cached eep153_tools-0.12.4-py2.py3-none-any.whl (4.9 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached python_gnupg-0.5.4-py2.py3-none-any.whl.metadata (2.0 kB)
Using cached python_gnupg-0.5.4-py2.py3-none-any.whl (21 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached gspread_pandas-3.3.0-py2.py3-none-any.whl.metadata (10 kB)
Using cached gspread_pandas-3.3.0-py2.py3-none-any.whl (27 kB)
  Attempting uninstall: gspread_pandas
    Found existing installation: gspread-pandas 2.2.3
    Uninstalling gspread-pandas-2.2.3:
      Successfully uninstalled gspread-pandas-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
def format_id(id,zeropadding=0):
    """Nice string format for any id, string or numeric.

    Optional zeropadding parameter takes an integer
    formats as {id:0z} where
    """
    if pd.isnull(id) or id in ['','.']: return None

    try:  # If numeric, return as string int
        return ('%d' % id).zfill(zeropadding)
    except TypeError:  # Not numeric
        return id.split('.')[0].strip().zfill(zeropadding)
    except ValueError:
        return None

data_url = "https://docs.google.com/spreadsheets/d/12Z4n8HbFZRYvH6B-D8EDLDibRiL50zNMlSBLMJ41C1o/"

In [3]:
import pandas as pd
from eep153_tools.sheets import read_sheets

recipes = read_sheets(data_url, sheet="recipes")
recipes = (recipes
           .assign(parent_foodcode = lambda df: df["parent_foodcode"].apply(format_id),
                   ingred_code = lambda df: df["ingred_code"].apply(format_id))
           .rename(columns={"parent_desc": "recipe"}))


# lets see an example of a recipe.
recipes[recipes["recipe"].str.contains("Rice", case=False)]

,parent_foodcode,recipe,ingred_code,ingred_desc,ingred_wt
229,13210260,"Rice flour cream, Puerto Rican style",1077,"Milk, whole, 3.25% milkfat, with added vitamin D",488.000
230,13210260,"Rice flour cream, Puerto Rican style",2047,"Salt, table",4.500
231,13210260,"Rice flour cream, Puerto Rican style",9156,"Lemon peel, raw",1.000
232,13210260,"Rice flour cream, Puerto Rican style",19335,"Sugars, granulated",62.500
233,13210260,"Rice flour cream, Puerto Rican style",20044,"Rice, white, long-grain, regular, raw, enriched",69.375
...,...,...,...,...,...
34450,77316510,"Stuffed cabbage, with meat and rice, Syrian di...",14411,"Beverages, water, tap, drinking",355.500
34451,77316510,"Stuffed cabbage, with meat and rice, Syrian di...",20044,"Rice, white, long-grain, regular, raw, enriched",92.500
34452,77316510,"Stuffed cabbage, with meat and rice, Syrian di...",23222,"Ground beef, raw, averaged",113.400
34933,91721000,Licorice,19108,"Candies, jellybeans",100.000


In [4]:
nutrition = (read_sheets(data_url, sheet="nutrients")
             .assign(ingred_code = lambda df: df["ingred_code"].apply(format_id)))

display(nutrition.head())
nutrition.columns
nutrition.shape

,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,Vitamin B12,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc
0,1001,"Butter, salted",2.529,2.587,7.436,21.697,0.961,9.999,19.961,2.728,...,0.17,0.0,0.003,0.0,0.0,2.32,0.0,7.0,15.87,0.09
1,1002,"Butter, whipped, with salt",2.039,2.354,7.515,20.531,1.417,7.649,17.370,2.713,...,0.07,0.0,0.008,0.0,0.0,1.37,0.0,4.6,16.72,0.05
2,1003,"Butter oil, anhydrous",2.495,2.793,10.005,26.166,2.228,12.056,25.026,2.247,...,0.01,0.0,0.001,0.0,0.0,2.80,0.0,8.6,0.24,0.01
3,1004,"Cheese, blue",0.601,0.491,3.301,9.153,0.816,3.235,6.622,0.536,...,1.22,0.0,0.166,0.0,0.5,0.25,0.0,2.4,42.41,2.66
4,1005,"Cheese, brick",0.585,0.482,3.227,8.655,0.817,3.455,7.401,0.491,...,1.26,0.0,0.065,0.0,0.5,0.26,0.0,2.5,41.11,2.60


(2750, 67)

In [5]:
# normalize weights to percentage terms. 
recipes['ingred_wt'] = recipes['ingred_wt']/recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
df = recipes.merge(nutrition, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(df.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
df[numeric_cols] = df[numeric_cols].mul(df["ingred_wt"], axis=0)

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. The ** effectively "unpacks" the key value pairs in each dictionary
df = df.groupby('parent_foodcode').agg({**{col: "sum" for col in numeric_cols},
                                        "recipe": "first"})

df.index.name = "recipe_id"

food_names = df["recipe"]
print(food_names.head())
df.head()

recipe_id
11000000                       Milk, human
11100000                         Milk, NFS
11111000                       Milk, whole
11111100           Milk, low sodium, whole
11111150    Milk, calcium fortified, whole
Name: recipe, dtype: object


,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,Linolenic Acid,Stearidonic acid,...,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc,recipe
recipe_id,,,,,,,,,,,,,,,,,,,,,
11000000,0.06300,0.2560,0.32100,0.91900,0.12900,0.29300,1.475,0.3740,0.052,0.0,...,0.0,0.011,5.00,0.100,0.08,0.0,0.30,87.500,0.1700,"Milk, human"
11100000,0.03825,0.0405,0.14275,0.42475,0.01175,0.18575,0.397,0.0535,0.022,0.0,...,0.0,0.037,0.05,1.225,0.03,0.0,0.15,89.525,0.4225,"Milk, NFS"
11111000,0.07500,0.0770,0.29700,0.82900,0.00000,0.36500,0.812,0.1200,0.075,0.0,...,0.0,0.036,0.00,1.300,0.07,0.0,0.30,88.130,0.3700,"Milk, whole"
11111100,0.08700,0.0970,0.34800,0.91000,0.07700,0.41900,0.870,0.0780,0.050,0.0,...,0.0,0.034,0.90,1.300,0.08,0.0,0.30,88.200,0.3800,"Milk, low sodium, whole"
11111150,0.07500,0.0770,0.29700,0.82900,0.00000,0.36500,0.812,0.1200,0.075,0.0,...,0.0,0.036,0.00,1.300,0.07,0.0,0.30,88.130,0.3700,"Milk, calcium fortified, whole"


In [6]:
prices = read_sheets(data_url, sheet="prices")[["food_code", "year", "price"]]

prices["food_code"] = prices["food_code"].apply(format_id)

prices = prices.set_index(["year", "food_code"])
print(prices.index.levels[0])

# we'll focus on the latest price data
prices = prices.xs("2017/2018", level="year")

# drop rows of prices where the price is "NA"
prices = prices.dropna(subset="price")

print(f"We have prices for {prices.shape[0]} unique recipes (FNDDS food codes)")

Index(['2011/2012', '2013/2014', '2015/2016', '2017/2018'], dtype='object', name='year')
We have prices for 4435 unique recipes (FNDDS food codes)


In [7]:
rda = read_sheets(data_url, sheet="rda")

rda = rda.set_index("Nutrient")

rda.columns, rda.head()

(Index(['Nutrient Type', 'Unit', 'Constraint Type', 'Child_1_3', 'Female_4_8',
        'Male_4_8', 'Female_9_13', 'Male_9_13', 'Female_14_18', 'Male_14_18',
        'Female_19_30', 'Male_19_30', 'Female_31_50', 'Male_31_50',
        'Female_51U', 'Male_51U'],
       dtype='object'),
               Nutrient Type  Unit Constraint Type  Child_1_3  Female_4_8  \
 Nutrient                                                                   
 Energy                Macro  kcal             RDA     1000.0      1200.0   
 Protein               Macro     g             RDA       13.0        19.0   
 Carbohydrate          Macro     g             RDA      130.0       130.0   
 Dietary Fiber         Macro     g             RDA       14.0        16.8   
 Linoleic Acid         Macro     g              AI        7.0        10.0   
 
                Male_4_8  Female_9_13  Male_9_13  Female_14_18  Male_14_18  \
 Nutrient                                                                    
 Energy           1

In [8]:
common_recipes = df.index.intersection(prices.index)

# python tip: given a list of indices, "loc" both subsets and sorts. 
df = df.loc[common_recipes]
prices = prices.loc[common_recipes]

# lets remap the price dataframe index to be the actual food names.
prices.index = prices.index.map(food_names)

A_all = df.T

print(prices.head())
print(A_all.head())

                           price
Milk, NFS               0.100484
Milk, whole              0.09828
Milk, reduced fat (2%)  0.092085
Milk, low fat (1%)      0.090914
Milk, fat free (skim)   0.092441
                 11100000 11111000 11112110 11112210 11113000 11114300  \
Capric acid       0.03825    0.075    0.049    0.027    0.002    0.027   
Lauric acid        0.0405    0.077    0.055    0.029    0.001    0.029   
Myristic acid     0.14275    0.297    0.175    0.091    0.008    0.091   
Palmitic acid     0.42475    0.829    0.558    0.287    0.025    0.287   
Palmitoleic acid  0.01175      0.0    0.027    0.017    0.003    0.017   

                 11114330 11114350 11115000 11115100  ... 95312410 95312560  \
Capric acid         0.049    0.075    0.022    0.022  ...      0.0      0.0   
Lauric acid         0.055    0.077    0.025    0.025  ...      0.0      0.0   
Myristic acid       0.175    0.297    0.089    0.089  ...      0.0      0.0   
Palmitic acid       0.558    0.829    0.2

In [21]:
# pick a demographic (column from rda dataframe)
'''
select from 
['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 
'Female_14_18', 'Male_14_18','Female_19_30', 'Male_19_30', 
'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']
'''
group = "Male_31_50"

# create lower bounds and upper bounds.
bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI']), group]
bmax = rda.loc[rda['Constraint Type'].isin(['UL']), group]

# reindex ensures we only keep nutrients in bmin/bmax
Amin = A_all.reindex(bmin.index).dropna(how='all')
Amax = A_all.reindex(bmax.index).dropna(how='all')

b = pd.concat([bmin, -bmax])
A = pd.concat([Amin, -Amax])

#python tip: by typing "=" after the variable name inside the curly braces, it formats the output so we don't have to write f"variable = {variable}"
print(f"{bmin.shape=}")
print(f"{Amin.shape=}")
print(f"{bmax.shape=}")
print(f"{Amax.shape=}")
print(f"{b.shape=}")
print(f"{A.shape=}")
print(f"{prices.shape=}")

bmin.shape=(26,)
Amin.shape=(26, 4426)
bmax.shape=(1,)
Amax.shape=(1, 4426)
b.shape=(27,)
A.shape=(27, 4426)
prices.shape=(4426, 1)


In [22]:
from  scipy.optimize import linprog as lp
import numpy as np
p = prices
tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros
result = lp(p, -A, -b, method='highs')

result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 2.513002467156168
              x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            nit: 30
          lower:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 1.032e-02  8.658e-03 ...  8.587e-02
                              4.765e-01]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  2.443e+01 ...  3.052e+02
                              0.000e+00]
                 marginals: [-1.415e-04 -0.000e+00 ... -0.000e+00
                             -6.150e-06]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_

In [23]:
print(f"Cost of diet for {group} is ${result.fun:.2f} per day.")

Cost of diet for Male_31_50 is $2.51 per day.


In [24]:
# lets mess with the index on price df so they are recipe names not ids.

# get the result x in a series with food names
diet = pd.Series(result.x,index=prices.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(round(diet[diet >= tol], 2))


You'll be eating (in 100s of grams or milliliters):
Milk, low fat (1%)                             6.44
Carp, steamed or poached                       0.08
Egg, yolk only, raw                            0.25
Split peas, from dried, fat added              5.68
Rice, white, cooked, made with margarine       1.30
Cereal, rice flakes                            0.19
Cereal, toasted oat                            0.08
Wheat bran, unprocessed                        0.01
Ripe plantain, raw                             3.11
Mustard greens, fresh, cooked, no added fat    0.24
dtype: float64


In [19]:
tab = pd.DataFrame({"Outcome":A.to_numpy()@diet.to_numpy(),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)


With the following nutritional outcomes of interest:
                    Outcome  Recommendation
Nutrient                                   
Energy               2400.0          2400.0
Protein           71.806921            56.0
Carbohydrate     310.093265           130.0
Dietary Fiber     44.355546            33.6
Linoleic Acid     34.274963            17.0
Linolenic Acid     3.665249             1.6
Calcium         1041.727184          1000.0
Iron              20.112104             8.0
Magnesium         413.07181           400.0
Phosphorus      1452.478806           700.0
Potassium            4700.0          4700.0
Zinc                   11.0            11.0
Copper             1.312792             0.9
Selenium          66.589038            55.0
Vitamin A        1735.00444           900.0
Vitamin E         21.018864            15.0
Vitamin D              15.0            15.0
Vitamin C              90.0            90.0
Thiamin            1.877899             1.2
Riboflavin         2.3

In [20]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())


Constraining nutrients are:
['Energy', 'Potassium', 'Zinc', 'Vitamin D', 'Vitamin C', 'Niacin', 'Choline']
